### Задание
ДЗ - обучить при помощи tf (keras) API любое ДЗ по обучениею классических моделей. 
Можно сдавать на торче.

### Анализ
Интересно построить ансамбль моделей и изучить результат работы.  
Для разнообразия моделей активационные функции и кожффициент дропаута будут выбираться случайным образом.  
В качестве датасета был выбран MNIST

### Имплементация
Определим нашу модель

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np
from torchinfo import summary
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class CNN_Number_Predictor(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN_Number_Predictor, self).__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            self.random_activation(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            self.random_activation(),
            nn.Conv2d(32, 32, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(32),
            self.random_activation(),
            self.random_dropout(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            self.random_activation(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            self.random_activation(),
            nn.Conv2d(64, 64, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(64),
            self.random_activation(),
            self.random_dropout(),
            nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(128),
            self.random_activation(),
            nn.Flatten(),
            self.random_dropout(),
            nn.Linear(2048, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.seq(x)
        return x
    
    def random_activation(self):
        acts = [
                nn.ReLU(),
                nn.LeakyReLU(negative_slope=random.choice([0.01, 0.1, 0.3])),
                nn.GELU(),
                nn.SiLU(),
                nn.Mish(),
                nn.CELU(alpha=random.uniform(0.1, 1.0))
            ]
        return random.choice(acts)

    def random_dropout(self):
        return nn.Dropout(p=random.uniform(0.2, 0.7))

Выведем информацию о 1 модели

In [2]:
model = CNN_Number_Predictor().to(device)

summary(model, input_size=(1, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
CNN_Number_Predictor                     [1, 10]                   --
├─Sequential: 1-1                        [1, 10]                   --
│    └─Conv2d: 2-1                       [1, 32, 28, 28]           320
│    └─BatchNorm2d: 2-2                  [1, 32, 28, 28]           64
│    └─Mish: 2-3                         [1, 32, 28, 28]           --
│    └─Conv2d: 2-4                       [1, 32, 28, 28]           9,248
│    └─BatchNorm2d: 2-5                  [1, 32, 28, 28]           64
│    └─CELU: 2-6                         [1, 32, 28, 28]           --
│    └─Conv2d: 2-7                       [1, 32, 14, 14]           25,632
│    └─BatchNorm2d: 2-8                  [1, 32, 14, 14]           64
│    └─Mish: 2-9                         [1, 32, 14, 14]           --
│    └─Dropout: 2-10                     [1, 32, 14, 14]           --
│    └─Conv2d: 2-11                      [1, 64, 14, 14]           18,496
│  

Настроим загрузчики данных и аугментацию. Для аугментации будем испоьлозвать случайные афинные преобразования и стирания части информации.

In [2]:
train_transform = transforms.Compose([
    transforms.RandomAffine(degrees=(-20, 20), translate=(0.2, 0.2), shear=10, scale=(0.5, 1.2)),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.25),
    transforms.Normalize((0.1307,), (0.3081,)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

training_data = MNIST(
    root="data",
    train=True,
    download=True,
    transform=train_transform
)

test_data = MNIST(
    root="data",
    train=False,
    download=True,
    transform=test_transform
)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, num_workers=16)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True, num_workers=16)

Using downloaded and verified file: data/MNIST/raw/train-images-idx3-ubyte.gz
Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Using downloaded and verified file: data/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:00<00:00, 2088815.75it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 15035934.31it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



Определим функцию для обучения 1 модели и функцию тестирования ансамбля

In [3]:
loss_fn = nn.CrossEntropyLoss()

def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for _ , (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test(dataloader, models, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    majority_votes = []

    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        predictions = []

        for model in models:
            model.eval()
            with torch.no_grad():
                pred = model(X)
                predictions.append(pred)

        predictions = torch.stack(predictions)
        majority_vote = torch.mode(predictions.argmax(dim=2), dim=0).values
        majority_votes.append(majority_vote)

        loss = loss_fn(predictions.mean(dim=0), y)
        test_loss += loss.item()
        correct += (majority_vote == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    return 100 * correct

Обучим 32 модели и выведем Accuracy для тестового датасета.

In [4]:
num_models = 32
models = [CNN_Number_Predictor().to(device) for _ in range(num_models)]
optimizers = [torch.optim.AdamW(model.parameters()) for model in models]

for i, model in enumerate(models):
    for _ in range(100):
        train(train_dataloader, model, loss_fn, optimizers[i])
    torch.save(model.state_dict(), f'ensemble_model_{i}_weights.pth')
print(test(test_dataloader, models, loss_fn))

99.48


### Выводы
- Обучен ансамбль из 32 моделей
- Использована аугментация данных для лучшей обобщающей способности модели
- Использован оптимизатор AdamW со штрафом за большие весовые коэффициенты
- Accuracy модели 99.48%
